In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [8]:
%matplotlib inline
from pathlib import Path
import itertools
from dataclasses import dataclass, fields

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams 

from matplottoy.data import iris
from matplottoy.encoders import color, position
from matplottoy.artists import point, bar, utils


spath = Path("../../../draft/figures/code/")
rcParams['figure.dpi'] = 200

In [9]:
data = {'fruit': ['apple', 'orange', 'lemon', 'lime'], 
       'calories':[95, 67, 17, 20], 
       'juice':[True, True, False, False]}
df = pd.DataFrame(data)

In [14]:
data = iris.DataFrame(df)

In [19]:
aes = {'position': ['fruit'],
       'length': ['calories'],
       'floor': None, 
       'width': None,
       'facecolors': ['juice'],
       'edgecolors': None,
       'linewidths' None,
       'linestyle': None}


,fruit,calories,juice
0,apple,95,True
1,orange,67,True
2,lemon,17,False
3,lime,20,False


In [21]:
position = position.Identity()
mapping = {'apple': 0, 'orange': 2, 'lemon': 4, 'lime': 6}
length = position.Nominal(mapping)
color = color.Nominal({True:orange, False:blue})
floor  = 
width =
facecolor=
edgecolor=
linewidth = 
linestyle = 
       


In [23]:
dict(zip(df['fruit'].to_list(), range(0,10,2)))

{'apple': 0, 'orange': 2, 'lemon': 4, 'lime': 6}